## Single glacier runoff series for multiple GCMs

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from cycler import cycler
import datetime 

### Define the case we'll study
We will need to make some consistent selections below: which basin, which SSP, which GCM to compare the two glacier models' output.

In [ ]:
test_glacier = 'RGI60-11.00181'

all_ssps = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
which_ssp = all_ssps[1]   #specifying the SSP

#All of the climate models used in GloGEM
modelnames = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0']
which_model = modelnames[0] ## specify one model (for GloGEM format)

# PyGEM has 12 GCM forcings as well; TODO: confirm they are in the same order as above.  Until then, use BCC-CSM2-MR
which_gcm = 1 ## applies to PyGEM

In [ ]:
## define local file paths
fstem_pygem = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/PyGEM/11/R11_runoff_monthly_c2_ba1_1set_2000_2100-{}-Batch-'.format(which_ssp)
fpath_glogem_huss = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/GloGEM-output/'
# fpath_oggm = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/OGGM/{}/'.format(test_basin)

Define which batch has our ID of interest, which you can read from the last 4 digits of the RGI ID.

In [ ]:
which_batch = '1-1000.nc'

ds_pygem = xr.open_dataset(fstem_pygem+which_batch)

In [ ]:
tg = int(test_glacier.replace('RGI60-11.00', '')) ## transform test glacier ID into an index within this batch
single_glacier_pygem = ds_pygem.glac_runoff_monthly.sel(glacier=tg).resample(time='AS').sum()

In [ ]:
total_models = 13
color_spacing = np.linspace(0,1, num=total_models)
line_colors = plt.colormaps['magma'](color_spacing)

fig, ax = plt.subplots()
for i in range(1,13):
    to_plot = single_glacier_pygem.sel(model=i).rolling(time=15, min_periods=10).mean()
    ax.plot(to_plot.time.values, to_plot, color=line_colors[i], label=to_plot.Climate_Model.values)
ax.legend(loc='best')
ax.set(xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      ylim = (4.75e6, 8.5e6),
       xlabel='Year',
       ylabel='Annual runoff [m3]',
       title='PyGEM, {}'.format(which_ssp)
      )
# single_glacier_pygem.plot.line(hue='model')

### GloGEM values for comparison

In [ ]:
df_disc_huss = pd.read_csv(fpath_glogem_huss +'Discharge-20230124/{}/RGIreg11_Discharge_individual.dat'.format(which_ssp), 
                           sep='\s+', header=0, skiprows=1, index_col=0)
df_disc_huss.columns = pd.to_datetime(df_disc_huss.columns)

#Read glacier area (INITIAL) to a pandas dataframe
df_area_huss = pd.read_csv(fpath_glogem_huss+'/Area-20230124/{}/RGIreg11_Area_individual.dat'.format(which_ssp), 
                           sep='\s+', header=0, skiprows=1, index_col=0)
df_area_huss.columns = pd.to_datetime(df_area_huss.columns)
df_area_init = df_area_huss.loc[:,'1980-01-01'] ## select just the initial area to compute basin runoff

In [ ]:
tg_huss = float(test_glacier.replace('RGI60-', ''))
single_glacier_glogem_huss = df_disc_huss.loc[tg_huss] ## select the rows with IDs found in the list
## allowing pandas to use the RGI IDs as a float for now...TODO - confirm this will work okay

annual_singleglac = single_glacier_glogem_huss.transpose().resample('AS').sum() ## annual total discharge per glacier
annual_singleglac

In [ ]:
df_area_single = df_area_init.loc[tg_huss]

In [ ]:
single_glac_glogem_runoff = annual_singleglac.mul(df_area_single)[:-1]

In [ ]:
# single_runoff_total_gt = 1e-3 * single_glac_glogem_runoff[:-1]

In [ ]:
fig, ax = plt.subplots()
single_glac_glogem_runoff.plot(ax=ax)
ax.set(xlabel = 'Year', ylabel='Annual runoff [m3]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1))) ## visualize runoff to the example basin from GloGEM

In [ ]:
df_disc_huss

In [ ]:
df_disc_huss_raw = pd.read_csv(fpath_glogem_huss +'Discharge-20230124/{}/RGIreg11_Discharge_individual.dat'.format(which_ssp), 
                           sep='\s+', header=0, skiprows=1, index_col=0)

In [ ]:
df_disc_huss_raw

### Raw GloGEM read-in, following Finn
TODO: incorporate Finn's script via Henson remote access